<a href="https://colab.research.google.com/github/fishenzone/portfolio/blob/main/projects/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# !pip install tensorflow-datasets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from keras.preprocessing import sequence

# !wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
# !wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

--2022-04-30 20:11:24--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K   782KB/s    in 0.4s    

2022-04-30 20:11:25 (782 KB/s) - ‘train-data.tsv’ saved [358233/358233]

--2022-04-30 20:11:25--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[===============

In [ ]:
def reader_data(file_name):
  data = pd.read_csv(
      file_name, 
      sep='\t', 
      header=None,
      names=['label', 'text'])
  return data

train_data = reader_data('train-data.tsv')
valid_data = reader_data('valid-data.tsv')

print(f'TRAIN DATA \n {train_data.tail()} \n\n VALID DATA \n {valid_data.tail()} \n\n TRAIN DATA SUMMARY')

print(train_data.info(), '\n\n TRAIN DATA SUM of NULL')
print(train_data.isnull().sum(), '\n\n VALID DATA SUMMARY')
print(valid_data.info(), '\n\n TEST DATA SUM of NULL')
print(valid_data.isnull().sum())

TRAIN DATA 
      label                                               text
4174   ham  just woke up. yeesh its late. but i didn't fal...
4175   ham  what do u reckon as need 2 arrange transport i...
4176  spam  free entry into our £250 weekly competition ju...
4177  spam  -pls stop bootydelious (32/f) is inviting you ...
4178   ham  tell my  bad character which u dnt lik in me. ... 

 VALID DATA 
      label                                               text
1387   ham  true dear..i sat to pray evening and felt so.s...
1388   ham               what will we do in the shower, baby?
1389   ham  where are you ? what are you doing ? are yuou ...
1390  spam  ur cash-balance is currently 500 pounds - to m...
1391  spam  not heard from u4 a while. call 4 rude chat pr... 

 TRAIN DATA SUMMARY
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4179 entries, 0 to 4178
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   4179 non-null 

In [ ]:
def preprocess(data):
  X = np.array(data.iloc[:,1])
  y = data.iloc[:,0].astype('category').cat.codes
  return X, y

X_train, y_train = preprocess(train_data)
X_valid, y_valid = preprocess(valid_data)
encoder = keras.layers.TextVectorization()
encoder.adapt(X_train)

In [ ]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.1, patience=3), 
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.001, patience=5)
]

model.compile(optimizer='adam', metrics=['accuracy'], loss='binary_crossentropy')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 64)          523584    
                                                                 
 bidirectional (Bidirectiona  (None, 128)              66048     
 l)                                                              
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dense_1 (Dense)             (None, 128)               8320      
                                                                 
 dense_2 (Dense)             (None, 1)                 1

In [ ]:
history = model.fit(
    x = X_train,
    y = y_train, 
    epochs=50,
    validation_data = (X_valid, y_valid),
    callbacks = callbacks
)

Epoch 1/50
131/131 [==============================] - 23s 66ms/step - loss: 0.2125 - accuracy: 0.9289 - val_loss: 0.0434 - val_accuracy: 0.9849 - lr: 0.0010
Epoch 2/50
131/131 [==============================] - 6s 42ms/step - loss: 0.0189 - accuracy: 0.9952 - val_loss: 0.0529 - val_accuracy: 0.9820 - lr: 0.0010
Epoch 3/50
131/131 [==============================] - 6s 44ms/step - loss: 0.0103 - accuracy: 0.9964 - val_loss: 0.0354 - val_accuracy: 0.9899 - lr: 0.0010
Epoch 4/50
131/131 [==============================] - 6s 43ms/step - loss: 0.0010 - accuracy: 0.9998 - val_loss: 0.0429 - val_accuracy: 0.9871 - lr: 0.0010
Epoch 5/50
131/131 [==============================] - 5s 40ms/step - loss: 2.7992e-04 - accuracy: 1.0000 - val_loss: 0.0438 - val_accuracy: 0.9885 - lr: 0.0010
Epoch 6/50
131/131 [==============================] - 6s 44ms/step - loss: 1.8888e-04 - accuracy: 1.0000 - val_loss: 0.0455 - val_accuracy: 0.9885 - lr: 0.0010
Epoch 7/50
131/131 [==============================] - 6

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  prediction = model.predict(np.array([pred_text])).item()
  if prediction < 0.5:
    pred_list = [f'{prediction:.2e}', 'ham']
  else:
    pred_list = [f'{prediction:.2e}', 'spam']
  return pred_list

pred_text = 'how are you doing today'

prediction = predict_message(pred_text)
print(prediction)

['1.95e-06', 'ham']


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


You passed the challenge. Great job!
